In [1]:
# import dependencies
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import numpy as np

In [2]:
def scrape_data(url_list):
    
    # import dependencies
    from bs4 import BeautifulSoup
    import requests
    import re
    import timeit
    import pandas as pd
    
    
    #beginning the timer
    start = timeit.default_timer()

    # create variables for player stats
    # make them global so that they can be accessed outside of the func
    global name
    name = []
    global team_abrev
    team_abrev = []
    global game_id_player
    game_id_player = []
    global game_map_player
    game_map_player = []
    global adr
    adr = []
    global first_blood_both
    first_blood_both =[]
    global first_blood_t
    first_blood_t =[]
    global first_blood_ct
    first_blood_ct =[]
    global first_death_both
    first_death_both = []
    global first_death_t
    first_death_t = []
    global first_death_ct
    first_death_ct = []
    global agent
    agent = []

    # create variables for match stats
    # make them global so that they can be accessed outside of the func
    global game_id_match
    game_id_match = []
    global game_map_match
    game_map_match = []

    #############################################################################  
    print('Using provided URL list')
    print('------------------------')

    url_list = url_list  # assuming url_list is passed as an argument

    print('------------------------')
    print(f'Found {len(url_list)} games!\n')

    ############################################################################# 

    print('Beginning Stat Retrieval')
    print('------------------------')
    
    # loop through the urls
    for idx, url in enumerate(url_list):
        
        try:
            # get the html file using request
            html_txt = requests.get(url)
            soup = BeautifulSoup(html_txt.text, 'lxml')
            game = soup.find('div', class_='vm-stats-game')
            table_info = game.find('table', class_='wf-table-inset mod-overview')
        except AttributeError:
            print(f'No game data for game {idx}')

        game_maps = len(soup.find_all('div', class_='vm-stats-game'))
    
    ############################################################################# 

        z=0
        for x in range(game_maps-1):
            # setting the location for mapping
            body = table_info.find('tbody')
            tr = body.find_all('tr')

            # temporary variables
            a=[]
            # create lists to hold each string of information
            name_web=[]
            team_web=[]
            team_close=[]

            # map
            game_header = game.find('div', class_='vm-stats-game-header')
            div_map = game_header.find('div', class_='map')
            div_style = div_map.find('div')
            span_map = div_style.find('span').text
            # remove the weird letters
            p = '\n\t*'
            p1 = '\t*'
            span_map = re.sub(p, '', span_map)
            span_map = re.sub(p1, '', span_map)
            # remove the word PICK from the string
            p2 = 'PICK'
            map_str= re.sub(p2, '', span_map)
            # fill the list with the map names
            i = 0
            while i<10:
                game_map_player.append(map_str)
                i+=1
            game_map_match.append(map_str)
            
    ############################################################################# 
    
            # all stats for the map 
            y=0
            while y < 2:
                # use a while loop to access each players info and send it into the list
                x=0
                while x < len(tr):
                    trx = tr[x]
                    # name and team_abrev
                    td_name = trx.find('td', class_='mod-player')
                    div = td_name.find('div')
                    a.append(div.find('a'))
                    # agent
                    try:
                        td_agent = trx.find('td', class_='mod-agents')
                        span_agent = td_agent.find('span', class_='stats-sq mod-agent small')
                        agent.append(span_agent.img.get('title'))
                    except AttributeError:
                        agent.append('not found')


                    x+=1

                # go to the next 'table' to find the stats of the other team
                if y == 0:
                    table_info = table_info.find_next('table', class_='wf-table-inset mod-overview')
                    body = table_info.find('tbody')
                    tr = body.find_all('tr')
                    
    ############################################################################# 
    
                # use a while loop to find the info that we need and put it into the lists
                i=0
                while i < len(a):
                    ai = a[i]
                    name_web.append(ai.find('div', class_='text-of').text)
                    team_web.append(ai.find('div', class_='ge-text-light').text)
                    i+=1

                # use regex to fix the names
                p = '\n\t*'
                p1 = '\t*'
                for n in name_web:
                    name.append(re.sub(p, '', n))

                for n in team_web:
                    team_close.append(re.sub(p, '', n))
                for b in team_close:
                    team_abrev.append(re.sub(p1, '', b))

                # clear temporary variables
                a = []
                name_web=[]
                team_web=[]
                team_close = []

                y+=1

                # mapping to the next maps
                if z == 0:
                    # map 2
                    # need to find_next because 'all' is the 2nd tab
                    game = game.find_next('div', class_='vm-stats-game')
                    game = game.find_next('div', class_='vm-stats-game')
                elif z == 1:
                    # map 3
                    game = game.find_next('div', class_='vm-stats-game')

                z+=x
    ############################################################################# 
    # finding the game id based on the length of the data we have
    
    i=0
    while i < len(team_abrev)/10:
        game_id_match.append(i)
        z=0
        while z < 10:
            game_id_player.append(i)
            z+=1
        i+=1
        
    print('------------------------')
    # ending the timer
    end = timeit.default_timer()
    time_sec = end - start
    time_sec = "{:.2f}".format(time_sec)
    print(f'Stats retrieved in {time_sec} seconds')
    
    return

In [3]:
# Function to read and print links from the file
def read_links(file_path):
    with open(file_path, 'r') as file:
        links = file.readlines()
        links = [link.strip() for link in links]
    return links

file_path = '/Users/bmac9/Desktop/Coding Stuff/Valorant_Project/Fantasy-Val/Data/over_list.txt'
# Access the links from the file
over_list = read_links(file_path)
print(f"Links read from file {file_path}")

Links read from file /Users/bmac9/Desktop/Coding Stuff/Valorant_Project/Fantasy-Val/Data/over_list.txt


In [4]:
scrape_data(over_list)
    
player_stat_dict = {
    'game_id': game_id_player,
    'player_name': name,
    'agent': agent,
    'team_abrev': team_abrev,
    'map': game_map_player
}

df = pd.DataFrame(player_stat_dict)
df

Using provided URL list
------------------------
------------------------
Found 25 games!

Beginning Stat Retrieval
------------------------
------------------------
Stats retrieved in 14.51 seconds


,game_id,player_name,agent,team_abrev,map
0,0,mazin,Omen,MIBR,Ascent
1,0,artzin,Yoru,MIBR,Ascent
2,0,ShahZaM,Sova,MIBR,Ascent
3,0,liazzi,Killjoy,MIBR,Ascent
4,0,Pa1nt,Iso,MIBR,Ascent
...,...,...,...,...,...
625,62,trent,not found,G2,TBD
626,62,valyn,not found,G2,TBD
627,62,JonahP,not found,G2,TBD
628,62,leaf,not found,G2,TBD


In [5]:
# Save the DataFrame to a CSV file
df.to_csv('../Data/name_map.csv', index=False)

print('DataFrame saved to name_map.csv')

DataFrame saved to name_map.csv
